## LOAD

In [1]:
%load_ext autotime
%load_ext autoreload
%autoreload 2
%matplotlib inline 
#inline
# %pdb
%config InlineBackend.figure_format ='retina'

import os
import pathlib
import sys

## setting path
## set the base path
# os.chdir(pathlib.Path(os.path.dirname(path[0])).parent.absolute())
# ## set path to sys for Lambda function
# sys.path.insert(0, os.getcwd())

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

time: 513 ms (started: 2023-04-03 16:42:42 +02:00)


In [2]:
# !zenml data-validator register ireland_data_validator --flavor=evidently

# # Register the MLflow experiment tracker
# !zenml experiment-tracker register ireland_green_tracker --flavor=mlflow --tracking_uri=http://0.0.0.0:7009 --tracking_token=resonanz_ireland

# !zenml orchestrator register ireland_green_orchestrator --flavor=airflow --local=True
# # --flavor=local
# # --flavor=airflow --local=True

# # zenml model-registry register ireland_model_registry --flavor=mlflow

# !zenml artifact-store register ireland_green_artifact_store --flavor=s3 --path=s3://bayaga/ireland

# # Register the MLflow model deployer
# !zenml model-deployer register ireland_green_dev --flavor=mlflow

# # !zenml feature-store register ida_feast_store --flavor=feast --feast_repo="s3://bayaga/registry.pb"

# # Create a new stack that includes an MLflow experiment
# !zenml stack register ireland_green_deployment_stack -e ireland_green_tracker \
#     -a ireland_green_artifact_store -o ireland_green_orchestrator -d ireland_green_dev \
#     -dv ireland_data_validator 
# #     -r ireland_model_registry
# #-f ida_feast_store

# # Set the new stack as active
!zenml stack set ireland_green_deployment_stack


Unable to find ZenML repository in your current working directory (/home/sourabh/Resonanz/Algotrader/ConvexDSC/algotrader/src/services/ireland/notebooks) or any parent directories. If you want to use an existing repository which is in a different location, set the environment variable 'ZENML_REPOSITORY_PATH'. If you want to create a new repository, run zenml init.
Running without an active repository root.
Connected to the ZenML server: 'http://127.0.0.1:8237'
Running with active workspace: 'default' (global)
⠦ Setting the global active stack to 'ireland_green_deployment_stack'.....
Error: "Stack 'ireland_green_deployment_stack' cannot be activated since it is not registered yet. Please register it first."
time: 4.65 s (started: 2023-04-03 16:42:43 +02:00)


## IMPORT

In [3]:
import pandas as pd
pd.options.plotting.backend = "plotly"
# import dtale
# from prophet import Prophet
from neuralprophet import NeuralProphet, set_log_level, set_random_seed
set_log_level("ERROR")
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import t
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
mlflow.set_tracking_uri("http://0.0.0.0:7009")
# from mlflow import MlflowClient 

import dtale

# from lightgbm import LGBMRegressor
# import dtale
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import power_transform, scale

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.inspection import DecisionBoundaryDisplay
from imblearn.ensemble import RUSBoostClassifier, EasyEnsembleClassifier, BalancedBaggingClassifier, BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score
from sklearn.metrics import explained_variance_score, max_error, r2_score, mean_absolute_error, f1_score

from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from imblearn.pipeline import Pipeline
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsRegressor, NeighborhoodComponentsAnalysis
# from lightgbm import LGBMClassifier
# import lightgbm as lgb
from sklearn.linear_model import *
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor, RandomForestRegressor, VotingClassifier
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsRegressor

from imblearn.over_sampling import SMOTE
from sklearn.calibration import CalibratedClassifierCV
import plotly.graph_objects as go
from statsmodels.miscmodels.ordinal_model import OrderedModel

from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import warnings

warnings.filterwarnings('ignore')

np.random.seed(0)


from absl import logging as absl_logging
import warnings

warnings.filterwarnings("ignore")
absl_logging.set_verbosity(-10000)

import mlflow
from zenml.steps import Output, step, StepContext


import sqlalchemy
import yaml
from typing import Dict, Union
from copy import deepcopy

from sklearn.base import ClassifierMixin
from sklearn.pipeline import Pipeline

import feast
from feast import FeatureStore
from feast.repo_config import RegistryConfig, RepoConfig
from feast.infra.online_stores.redis import RedisOnlineStoreConfig

from zenml.client import Client
from zenml.integrations.mlflow.flavors.mlflow_experiment_tracker_flavor import (
    MLFlowExperimentTrackerSettings,
)
from zenml.services import BaseService

from evidently.report import Report
from evidently.metric_preset import (
    DataQualityPreset,
    DataDriftPreset,
    ClassificationPreset,
)

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, DataQualityTestPreset
from evidently.tests import *

WARNING - (py.warnings._showwarnmsg) - /opt/conda/conda/envs/azba/lib/python3.9/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



distutils Version classes are deprecated. Use packaging.version instead.




time: 4.44 s (started: 2023-03-27 16:57:23 +02:00)


## FUNCTIONS

In [4]:
def hourlySample(data):
    d = data.copy()
    d.index = pd.DatetimeIndex(d.index) 
    return d.reindex(pd.date_range(start=d.index.min(), periods = 24 * len(d), freq='h'), method='ffill').squeeze()

def findOutliers(df):
    s = df.copy()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    iqr_lower = q1 - 8 * iqr
    iqr_upper = q3 + 8 * iqr
#     outliers = s[(s < iqr_lower) | (s > iqr_upper)]
    s[s < iqr_lower] = iqr_lower
    s[s > iqr_upper] = iqr_upper
    
    return s

def _scores(pred, true):
    display(confusion_matrix(true.loc[pred.index], pred))
    print("bAcc : ", bscore := balanced_accuracy_score(true.loc[pred.index], pred))
    print("acc :" , acc := accuracy_score(true.loc[pred.index], pred))
    print("f1 :", f1 := f1_score(true.loc[pred.index], pred))
    print("report :", classification_report(true.loc[pred.index], pred))
    print("precision, recall", prScore := precision_recall_fscore_support(true.loc[pred.index], pred, average='weighted'))
    
    return (bscore, acc, f1, prScore)

def _scoresArr(pred, true, printVal=True):
    bscore = balanced_accuracy_score(true, pred)
    acc = accuracy_score(true, pred)
    f1 = f1_score(true, pred)
    prScore = precision_recall_fscore_support(true, pred, average='weighted')
        
    if printVal:
        display(confusion_matrix(true, pred))
        print("bAcc : ", bscore)
        print("acc :" , acc)
        print("f1 :", f1)
        print("report :", classification_report(true, pred))
        print("precision, recall", )
    
    return (bscore, acc, f1, prScore)

def _scoresRegArr(actual, pred, printVal=True):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
        
    if printVal:
        print("rmse : ", rmse)
        print("mae : ", mae)
        print("r2 : ", r2)
        
    return (rmse, mae, r2)
    
def _toMultiIndex(d, name=None):
    reformed_dict = {}
    for outerKey, innerDict in d.items():
        for innerKey, values in innerDict.items():
            reformed_dict[(outerKey,
                           innerKey)] = values

    l = pd.DataFrame(reformed_dict)
    display(l)
    if name:
        l.to_csv(name + ".csv")
        
    return l

time: 40.6 ms (started: 2023-03-27 16:57:28 +02:00)


## DATA

### Fetch

In [8]:
@step
def getDates() -> Output(
    stDate = str, endDate=str, dts=tuple
):
    stDate = "2019-03-17"
    endDate = "2023-03-20" 
    
#     st = '2022-03-16'
    st = '2020-01-01'
    trEnd = '2022-06-30'
    tstStart = '2022-07-01'
    pEnd = '2023-02-20'
    
    dts = (st, trEnd, tstStart, pEnd)
    
    return stDate, endDate, dts

@step
def fetchData(_stDate:str, _endDate:str, context: StepContext)-> Output(dataFrame=pd.DataFrame):
    
#     repoConfig = RepoConfig(
#         registry=RegistryConfig(path="s3://bayaga/registry.pb"),
#         project="IDA",
#         provider="local",
#         offline_store="file",
#         online_store=RedisOnlineStoreConfig(connection_string="localhost:6379"),
#         entity_key_serialization_version=2,
#     )
#     store = FeatureStore(config=repoConfig)
    repoPath = "<path of feast repo>"
    store = FeatureStore(repo_path=repoPath)
    
    dd = [{'epoch_time' : int(pd.Timestamp(d.strftime('%Y-%m-%d %H:%M:%S')).timestamp())} for d in pd.date_range(start=_stDate, end=_endDate, freq='H')]

    batch_features = store.get_online_features(
        entity_rows=dd,
        features=store.get_feature_service("ireland_data_service", allow_cache=True),
    ).to_df()
    
    df = batch_features.copy()
    
    df.set_index('epoch_time', inplace=True)
    df.index = pd.to_datetime(df.index, unit='s')
    df.index.name = None
    
    dataFrame = df.astype(float)
    
    # drop the first NaN value as that might be because of shift
    dataFrame = dataFrame.iloc[dataFrame.notnull().all(axis=1).argmax() :]
    dataFrame = dataFrame.groupby(by=dataFrame.index, as_index=True).mean()
    
    dataFrame = dataFrame.reindex(
        pd.date_range(
            start=dataFrame.index.min(), end=dataFrame.index.max(), freq="H"
        ),
        fill_value=np.nan,
    )
    dataFrame = dataFrame.interpolate(method="bfill", axis=0)
    print(dataFrame.columns)
    return dataFrame
    

time: 82.5 ms (started: 2023-03-27 16:57:28 +02:00)


### Filter

In [9]:
# _stDate  = '2019-03-16'
@step
def filteredData(dataConcatenated : pd.DataFrame, _stDate : str) -> Output(targetSpreadWOOutlier=pd.DataFrame, features=pd.DataFrame):
    data = dataConcatenated.loc[_stDate:][:-1]
    
    targetSpread = data['price_diff'].dropna().asfreq('h')
    features = data.loc[targetSpread.index, ~data.columns.isin(['price_diff', 'ie_actual_wnd'])].asfreq('h')
    
    features = features.loc[: , ['day', 'weekday', 'year_day', 'week', 'month', 'quarter', 'lag1',
       'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag30']]
    
    targetSpreadWOOutlier = findOutliers(targetSpread).to_frame()
    
    return targetSpreadWOOutlier, features

time: 81.3 ms (started: 2023-03-27 16:57:28 +02:00)


### Transform

In [10]:
@step
def STLTransform(targetSpreadWOOutlier : pd.Series) -> Output(targetSpreadWOOutlierTrend=pd.Series):

    s = targetSpreadWOOutlier.copy().asfreq(freq='H')
    s.index = pd.to_datetime(s.index)

    stl_result = STL(s).fit()
    targetSpreadWOOutlierTrend = stl_result.trend
    stl_result.plot()
    
    return targetSpreadWOOutlierTrend

time: 78.4 ms (started: 2023-03-27 16:57:28 +02:00)


### Prepare

In [11]:
@step
def prepareSpreadTrend(targetSpreadWOOutlier : pd.Series, targetSpreadWOOutlierTrend: pd.Series, features : pd.DataFrame)-> Output(
        targetSpreadSign=pd.DataFrame, targetSpreadTrendSign=pd.DataFrame, targetSpreadDaySign=pd.DataFrame, features=pd.DataFrame, dayFeatures=pd.DataFrame, featureColumns=list
    ):

    targetSpreadReal = targetSpreadWOOutlier.to_frame().copy().rename(columns = {'price_diff' : 'spread_real'})
    targetSpreadSign = np.sign(targetSpreadReal.mask(targetSpreadReal <= 0, -1)).rename(columns = {'spread_real' : 'spread_sign'}).astype(int)
    
    #trend
    targetSpreadTrendReal = targetSpreadWOOutlierTrend.copy().to_frame('spread_trend_real')
    targetSpreadTrendSign = np.sign(targetSpreadTrendReal.mask(targetSpreadTrendReal <= 0, -1)).rename(columns = {'spread_trend_real' : 'spread_trend_sign'}).astype(int)

    
    targetSpreadDayReal = targetSpreadWOOutlier.to_frame().groupby([targetSpreadWOOutlier.index.date]).sum().rename(columns = {'price_diff' : 'spread_day_real'})
    targetSpreadDayReal.index = pd.to_datetime(targetSpreadDayReal.index)
    targetSpreadDaySign = np.sign(targetSpreadDayReal.mask(targetSpreadDayReal <= 0, -1)).rename(columns = {'spread_day_real' : 'spread_day_sign'}).astype(int)
    
    features = features.dropna()
    dayFeatures = features.resample('D', axis=0).sum() 
    
    featureColumns = list(features.columns)
    
    return targetSpreadSign, targetSpreadTrendSign, targetSpreadDaySign, features, dayFeatures, featureColumns

time: 97.8 ms (started: 2023-03-27 16:57:28 +02:00)


### Split

In [12]:
@step
def trainTestSplit(features:pd.DataFrame, targetSpreadSign:pd.DataFrame, targetSpreadTrendSign:pd.DataFrame, targetSpreadDaySign:pd.DataFrame, fetaureColumns:list, dates:tuple) -> Output(
        X_train=pd.DataFrame, X_test=pd.DataFrame, y_train=pd.DataFrame, y_test=pd.DataFrame, 
        y_trend_train=pd.DataFrame, y_trend_test=pd.DataFrame, 
        X_day_train=pd.DataFrame, X_day_test=pd.DataFrame, y_train_day_sign=pd.DataFrame, y_test_day_sign=pd.DataFrame
):

    st, trEnd, tstStart, pEnd = dates
    
    X_train, X_test = (features.loc[st : trEnd, fetaureColumns], features.loc[tstStart : pEnd, fetaureColumns])
    y_train, y_test = targetSpreadSign.loc[st : trEnd], targetSpreadSign.loc[tstStart : pEnd]
    y_trend_train, y_trend_test = targetSpreadTrendSign.loc[st : trEnd], targetSpreadTrendSign.loc[tstStart : pEnd]
    
    X_day_train, X_day_test= (features.loc[st : trEnd, fetaureColumns].resample('D', axis=0).sum(), 
                          features.loc[tstStart : pEnd, fetaureColumns].resample('D', axis=0).sum())
    y_train_day_sign, y_test_day_sign = targetSpreadDaySign.loc[st : trEnd], targetSpreadDaySign.loc[tstStart : pEnd]
    
    print(X_train)
    print(X_test)
    
    return X_train, X_test, y_train, y_test, y_trend_train, y_trend_test, X_day_train, X_day_test, y_train_day_sign, y_test_day_sign

time: 140 ms (started: 2023-03-27 16:57:29 +02:00)


## MODEL

### Train

#### Trend and Day 

In [13]:
@step
def getTrendModelAndMetadata()-> Output(
    modl=tuple, day=bool
):
    modl = ('trend_classifier',
                  BalancedBaggingClassifier(estimator=LogisticRegression(random_state=42,solver='lbfgs', max_iter=1000), random_state=0))
    day = False
    
    return modl, day

@step
def getDayModelAndMetadata()-> Output(
    modl=tuple, day=bool
):
    modl = ('day_classifier',
                  BalancedBaggingClassifier(estimator=LogisticRegression(random_state=42,solver='lbfgs', max_iter=1000), random_state=0))
    day = True
    
    return modl, day

@step(
    experiment_tracker="ireland_green_tracker",
    settings={
        "experiment_tracker.mlflow": MLFlowExperimentTrackerSettings(
            experiment_name="ireland_eval",
            nested=True,
            tags={"mlflow.runName": "evaluation"},
        )
    },
)
def trainModel(X:pd.DataFrame, Y:pd.DataFrame, modl:tuple, dayType:bool)-> Output(
    finalTrain = pd.DataFrame, model = Pipeline
):
    finalTrain = pd.DataFrame(index=Y.index)

    name, clf = modl

    p = [("transform", StandardScaler()), ("model", clf)]
    model = Pipeline(p)

    mlflow.set_tag("mlflow.runName", name)
    mlflow.sklearn.autolog(registered_model_name=f"{name}")

    model.fit(X.to_numpy(), Y.squeeze().to_numpy())

    score = model.score(X.to_numpy(), Y.squeeze().to_numpy())
    trainPred = model.predict(X.to_numpy())

    mlflow.log_metric("training score", score)

    if dayType:
        finalTrain["day_pred"] = hourlySample(
            pd.DataFrame(trainPred.astype(int), index=Y.index)
        )
    else:
        finalTrain["pred"] = trainPred.astype(int)

    return finalTrain, model

time: 135 ms (started: 2023-03-27 16:57:29 +02:00)


#### Combined

In [14]:
@step
def prepareCombinedFeatures(trendTrainPred:pd.DataFrame, dayTrainPred:pd.DataFrame, features:pd.DataFrame)-> Output(
    combinedFeatures=pd.DataFrame
):
    
    combinedFeatures = pd.concat([trendTrainPred, dayTrainPred, features], axis=1)
    
    return combinedFeatures

@step
def getCombinedModelAndMetadata()-> Output(
    modl=tuple
):
    modl = ('combined_classifier',
                  BalancedBaggingClassifier(estimator=CategoricalNB(fit_prior=False, force_alpha=True), random_state=0, verbose=False))
    
    return modl
            
#, experiment_tracker="ireland_green_tracker"
@step(enable_cache=False)
def trainCombinedModel(X:pd.DataFrame, Y:pd.DataFrame, modl:tuple)-> Output(
    finalTrain = pd.DataFrame, model = Pipeline
):
    datas = []
    
    finalTrain = pd.DataFrame(index = Y.index)
    
    name, clf = modl
            
    p = [
      ('transform', StandardScaler()),
      ('model', clf)
    ]
    model = Pipeline(p)

#     mlflow.sklearn.autolog(registered_model_name=f"{name}") 
    model.fit(X, Y.squeeze())

    score = model.score(X, Y.squeeze())
    trainPred = model.predict(X)
    
#     mlflow.sklearn.log_model(
#         model, f"models/{name}",
#         registered_model_name=f"{name}"
#     )
    
    finalTrain['combined_pred'] = trainPred.astype(int)
    
    return finalTrain, model, name

time: 154 ms (started: 2023-03-27 16:57:29 +02:00)


## REPORT

In [15]:
@step
def checkDataQuality(data:pd.DataFrame) -> pd.DataFrame:
    
    data_quality_report = TestSuite(tests=[
        TestNumberOfColumnsWithMissingValues(),
        TestNumberOfRowsWithMissingValues(),
        TestNumberOfConstantColumns(),
        TestNumberOfDuplicatedRows(),
        TestNumberOfDuplicatedColumns(),
        DataQualityTestPreset(),
#         DataStabilityTestPreset()
    ])
    dataQuality = data.rename(columns={"price_diff": "target"})
    data_quality_report.run(reference_data=None, current_data=dataQuality)
    data_quality_report.save_html("data_quality_report.html")
    
    return data

time: 154 ms (started: 2023-03-27 16:57:29 +02:00)


## PIPELINE

In [ ]:
from zenml.pipelines import pipeline
from zenml.integrations.mlflow.steps import (
    MLFlowDeployerParameters, 
    mlflow_model_deployer_step
)

@step
def deployment_decision() -> bool:
    return True

@pipeline(enable_cache=False)
def eval_pipeline(
    dates,
    importer,
    qualityChecker,
    filterData,
    transformer,
    prepare,
    splitter,
    modelAndMetadataTrend,
    modelAndMetadataDay,
    trendTrainer,
    dayTrainer,
    deploymentDecision, 
    trendModelDeployer,
    dayModelDeployer
):
    """Train and deploy a model with MLflow."""
    _stDate, _endDate, dts = dates()
    data = importer(_stDate, _endDate)
    qualityData = qualityChecker(data)
    
    target, features = filterData(qualityData, _stDate)
    trendTarget = transformer(target)
    targetSpreadSign, targetSpreadTrendSign, targetSpreadDaySign, features, dayFeatures, fetaureColumns = prepare(target, trendTarget, features)

    X_train, X_test, y_train, y_test, y_trend_train, y_trend_test, X_day_train, X_day_test, y_train_day_sign, y_test_day_sign = splitter(features, targetSpreadSign, targetSpreadTrendSign, targetSpreadDaySign, fetaureColumns, dts)
    
    modlTrend, dayTrend = modelAndMetadataTrend()
    trendTrainPred, trendModel = trendTrainer(X=X_train, Y=y_trend_train, modl=modlTrend, dayType=dayTrend)
    
    modlDay, day = modelAndMetadataDay()
    dayTrainPred, dayModel = dayTrainer(X=X_day_train, Y=y_train_day_sign, modl=modlDay, dayType=day)
    
    shouldDeploy = deploymentDecision()
    trendModelDeployer(shouldDeploy, trendModel)
    dayModelDeployer(shouldDeploy, dayModel)# new
    
eval_pipeline(
    dates=getDates(),
    importer=fetchData(),
    qualityChecker = checkDataQuality(),
    filterData = filteredData(),
    transformer = STLTransform(),
    prepare = prepareSpreadTrend(),
    splitter = trainTestSplit(), 
    modelAndMetadataTrend=getTrendModelAndMetadata(),
    modelAndMetadataDay=getDayModelAndMetadata(),
    trendTrainer = trainModel().configure('trendTrainer'),
    dayTrainer = trainModel().configure('dayTrainer'),
    deploymentDecision=deployment_decision(),
    trendModelDeployer=mlflow_model_deployer_step(
        MLFlowDeployerParameters(timeout=500, run_name='trend_classifier')
    ).configure('trendModelDeployer'),
    dayModelDeployer=mlflow_model_deployer_step(
        MLFlowDeployerParameters(timeout=500, run_name='day_classifier')
    ).configure('dayModelDeployer')
).run()


: 